# Generate dataset

The aim of this notebook is to create a varied dataset with atomRDF that can be used testing quering methods, and also to show to variety of tasks that can be done. This should be runnable top to bottom.

In [ ]:
%config IPCompleter.evaluation='unsafe'

Create a project

In [1]:
from pyiron_atomistics import Project
from atomrdf import KnowledgeGraph, System
import numpy as np

In [2]:
project = 'data_gen_1'
pr = Project(project)

In [3]:
kg = KnowledgeGraph(store='db', store_file=f'{project}.db')
kg.enable_workflow(pr, workflow_environment='pyiron')

Regular structures

In [ ]:
struct_Fe = System.create.element.Fe(graph=kg)
struct_Si = System.create.element.Si(graph=kg)
struct_l12 = System.create.lattice.l12(element=['Al', 'Ni'], 
                         lattice_constant=3.57, graph=kg)


In [ ]:
struct_gb_1 = System.create.defect.grain_boundary(axis=[0,0,1], 
                        sigma=5, 
                        gb_plane=[3, -1, 0],
                        element='Fe',
                        graph=kg)
struct_gb_2 = System.create.defect.grain_boundary(axis=[1,1,1], 
                        sigma=19, 
                        gb_plane=[-3, 2, 0],
                        element='Fe',
                        graph=kg)

struct_gb_3 = System.create.defect.grain_boundary(axis=[1,1,1], 
                        sigma=19, 
                        gb_plane=[-1, -1, -1],
                        element='Fe',
                        graph=kg)


In [ ]:
struct_Fe = System.create.element.Fe(graph=kg)
struct_with_vacancy = struct_Fe.delete(indices=[0], copy_structure=True)
struct_Al = System.create.element.Al(graph=kg)
struct_Al.substitute_atoms('Mg', indices=[0], copy_structure=True)

In [ ]:
slip_direction = np.array([1, 0, -1])
slip_plane = np.array([1, 1, 1])
slip_system = [slip_direction, slip_plane]
burgers_vector = 0.5
dislocation_line = np.array([1, 0, -1])
elastic_constant_dict = {'C11': 169, 'C12': 122, 'C44': 75.4}
sys = System.create.defect.dislocation(slip_system,
                                        dislocation_line,
                                        elastic_constant_dict,
                                        burgers_vector=burgers_vector,
                                        element='Cu',
                                        dislocation_type='monopole',
                                        graph=kg,
                                        )

In [ ]:
structure = pr.create.structure.annotated_structure.bulk('Cu', cubic=True)
job = pr.create.job.Lammps('j1', delete_existing_job=True, delete_aborted_job=True)
job.structure = structure
job.potential = '2001--Mishin-Y--Cu-1--LAMMPS--ipr1'
job.calc_md(pressure=0, temperature=500)
job.run()
kg.add_workflow(job, workflow_environment='pyiron')

In [ ]:
structure = pr.create.structure.annotated_structure.bulk('Cu', cubic=True)
ref_job = pr.create.job.Lammps('j1', delete_existing_job=True, delete_aborted_job=True)
ref_job.structure = structure
ref_job.potential = '2001--Mishin-Y--Cu-1--LAMMPS--ipr1'
ref_job.calc_minimize()
murn_job = ref_job.create_job(pr.job_type.Murnaghan, 'murn_job')
murn_job.input["num_points"] = 5
murn_job.run()
kg.add_workflow(murn_job, workflow_environment='pyiron', add_intermediate_jobs=True)

In [ ]:
struct = pr.create.structure.annotated_structure.bulk('Cu')
refjob = pr.create.job.Lammps('reflmp')
refjob.structure = struct
refjob.potential = '2009--Mendelev-M-I--Cu-Zr--LAMMPS--ipr1'
phono = pr.create.job.PhonopyJob('phono')
phono.ref_job = refjob
quasi = pr.create.job.QuasiHarmonicJob('quasi')
quasi.ref_job = phono
quasi.input["temperature_end"] = 500
quasi.input["temperature_steps"] = 2
quasi.input["axes"]=["x","y","z"]
quasi.input["strains"] = None
quasi.run()
kg.add_workflow(quasi, workflow_environment='pyiron')